In [10]:
# Загружаем библиотеки
import requests
import bs4
import re
import pandas as pd
import numpy as np


# Функция получения номера карточки (Property Record Card)
def get_card_number(parser):
    return re.findall("[0-9].*[0-9]",parser.find('table',cellpadding="1").text)    


# Функция получения таблицы Parcel Information
def get_parcel_info(parser, card_number):
    n_cols = 0
    n_rows = 0
    column_names = []
    
    # Отделяем нужную таблицу
    parc_info = parser.find_all('table', border="1", cellpadding="0", cellspacing="2", width="100%")[3]
    
    # Если есть заголовки столбцов, запоминаем их
    th_tags = parc_info.find_all('th')
    if len(th_tags) > 0 and len(column_names) == 0:
        for th in th_tags:
            column_names.append(th.get_text())
    
    # Находим число строк и столбцов
    # и заголовок (если он есть)
    for row in parc_info.find_all('tr'):
                
        # Находим количество строк
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows += 1
            if n_cols == 0:
                # Запоминаем количество столбцов
                n_cols = len(td_tags)
                        
        # Если есть заголовки столбцов, запоминаем их
        th_tags = row.find_all('th')
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text())
    
    # Формируем DataFrame
    columns = column_names if len(column_names) > 0 else range(0, n_cols)
    df = pd.DataFrame(columns = columns, index= range(0, n_rows))
    row_m = 0
    for row in parc_info.find_all('tr'):
        col_m = 0
        columns = row.find_all('td')
        for column in columns:
            
            cell_value = column.get_text().replace('\t','').replace('\n','').replace('\r','').replace('Sales Query','').strip()
            
            # Преобразовываем в числовой формат, что можно
            # Не преобразовываем то, что >= 11 символом - код
            if len(cell_value.replace(',', '').replace('$', '')) < 11:
                try:
                    cell_value = float(cell_value.replace(',', '').replace('$', ''))
                except ValueError:
                    pass
            
            df.iat[row_m, col_m] = cell_value
            
            col_m += 1
        if len(columns) > 0:
            row_m += 1
    
    # Заголовки - в первой строке, используем их
    if len(column_names) == 0:
        df = df.rename(columns=df.iloc[0]).drop(df.index[0])
 
    df['Property_Record_Card'] = card_number
            
    return df


# Список ссылок для парсинга
pages = ['http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=153114131800003010',
        'http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=163105162900021810',
        'http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=162906640000010420',
        'http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=162829126410051010',
        'http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=152824107510002074',
        'http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=152703758190000510',
        'http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=153114907950000104',
        'http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=162722716901220810',
        'http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=163116635220030020',
        'http://www.pcpao.org/?pg=http://www.pcpao.org/general.php?strap=152908546580011203']

# Просмотрим все страницы в цикле
for i, page in enumerate(pages):
    
    # Получаем код html-страницы - для анализа
    req = requests.get(page.replace('http://www.pcpao.org/?pg=',''))
    
    # Для обработки html-кода воспользуемся библиотекой Beautiful Soup 4
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    
    # Посмотрим на html-код страницы в отформатированном виде (с отступами)
    # (только для определения структуры страницы, потом не нужно)
    # print(parser.prettify())
    
    # Получаем номер карточки (Property Record Card)
    card_number = get_card_number(parser)[0]

    
    # Получаем таблицу Parcel Information и сохраняем в файл
    if i==0:
        df2 = get_parcel_info(parser, card_number)
    else:
        df2 = pd.concat([df2, get_parcel_info(parser, card_number)], ignore_index=True)  
    df2.to_csv('parcel_info.csv', index=False)

In [11]:
d = pd.read_csv('parcel_info.csv')

In [12]:
d.head()

,Most Recent Recording,Sales Comparison,Census Tract,Evacuation Zone (NOT the same as a FEMA Flood Zone),Flood Zone (NOT the same as your evacuation zone),Plat Book/Page,Property_Record_Card
0,19389/2143,221049.0,121030279041,A,Compare Preliminary to Current FEMA Maps,7/60,14-31-15-13180-000-3010
1,17505/0656,46343.0,121030248052,E,Compare Preliminary to Current FEMA Maps,2/27,05-31-16-16290-002-1810
2,20217/0154,69120.0,121030269123,NON EVAC,Compare Preliminary to Current FEMA Maps,4/28,06-29-16-64000-001-0420
3,14528/1630,141984.0,121030268114,NON EVAC,Compare Preliminary to Current FEMA Maps,74/119,29-28-16-12641-005-1010
4,17039/0648,177646.0,121030269082,NON EVAC,Compare Preliminary to Current FEMA Maps,99/88,24-28-15-10751-000-2074
